In [1]:
import os
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup as bs
import json
import datetime as dt
import time

from selenium import webdriver
from selenium.common.exceptions import TimeoutException

In [36]:
def get_main_html_source(show_alias):

    site = 'https://www.instagram.com/{0}/channel/?hl=en'.format(show_alias)
    
    browser = webdriver.Chrome(executable_path=r"C:\Users\Fang\Desktop\Python Trading\Trading\chromedriver.exe")

    browser.get(site)

    source_list = []

    source_list.append(browser.page_source)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    while True:

        curr_page_source = browser.page_source
        source_list.append(curr_page_source)

        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    browser.quit()
    
    return source_list

def ig_pull_init_function(source):
    curr_post_divs = bs(source, 'lxml').find('main').find_all("a")

    post_init_info = {'post_title': [],
                      'post_url': [],
                      'duration': []}

    for a in curr_post_divs:
        href = a['href'] 
        text = a.text
        if '/tv/' in href:
            try: 
                post_init_info['post_url'].append('https://www.instagram.com{}'.format(href))
                post_init_info['post_title'].append(a.find('div', attrs={'class': 'pu1E0'}).text)
                post_init_info['duration'].append(a.find('div', attrs={'class': 'zncDM'}).text)
            except:
                post_init_info['post_url'].append('https://www.instagram.com{}'.format(href))
                post_init_info['post_title'].append('N/A')
                post_init_info['duration'].append('N/A')

    return pd.DataFrame(post_init_info)
    
def get_post_info(curr_url):

    curr_post = bs(requests.get(curr_url).text, 'lxml')

    curr_post_json = json.loads(curr_post.find('script', type = 'application/ld+json').text.strip())

    alternate_name = curr_post_json['author']['alternateName']
    commentCount = curr_post_json['commentCount']
    
    try:
        caption = curr_post_json['caption']
    except:
        caption = ''
        
    likes = curr_post_json['description'].split(' Likes, ')[0].replace(',', '')
    views = curr_post_json['interactionStatistic']['userInteractionCount']
    uploadDate = curr_post_json['uploadDate']

    curr_post_df = pd.DataFrame({'alternate_name': alternate_name,
                                 'commentCount': commentCount,
                                 'caption': caption,
                                 'likes': likes,
                                 'views': views,
                                 'uploadDate': uploadDate}, index = [0])
    return curr_post_df

def get_show_info(show_alias):

    source_list = get_main_html_source(show_alias)

    source_post_links_df = pd.concat([ig_pull_init_function(source) for source in source_list], axis = 0).drop_duplicates().reset_index(drop = True)

    show_df = []

    for idx, row in source_post_links_df.iterrows():
        duration = row.duration
        title = row.post_title

        curr_url = row['post_url']

        curr_post_df = get_post_info(curr_url)

        curr_post_df['duration'] = duration
        curr_post_df['title'] = title
        curr_post_df['post_url'] = curr_url

        show_df.append(curr_post_df)

        print("Current Post: {0} --- Completed {1}%".format(title, round(idx/len(source_post_links_df)*100,2)))
        time.sleep(1)

    show_df = pd.concat(show_df, axis = 0).reset_index(drop = True)
    
    return show_df


In [ ]:
##################### ADD SHOW NAMES TO LIST HERE #########################

show_aliases = ['fallontonight', 'agt', 'nbcsongland', 'ninjawarrior', 'nbcbluffcitylaw', 'nbcbringthefunny', 
                'nbccouncilofdads', 'nbcindebted', 'nbclincoln','nbcperfectharmony','nbcsongland','nbcsunnyside','nbctheinbetween',
               'nbckenanshow','zoeysplaylist','nbcsnl','nbcbrooklyn99','nbcworldofdance','fallontonight',
               'nbcthisisus','nbcthegoodplace','nbcsvu','latenightseth','nbcsuperstore','ninjawarrior']

#################### LOOP TO CREATE DATAFRAME OF ALL DATA ##############################

all_shows_df = []

for i, show in enumerate(show_aliases):
    curr_show_data_df = get_show_info(show)
    all_shows_df.append(curr_show_data_df)
    print("Current Show: {0} --- Completed {1}%".format(show, round(i/len(show_aliases)*100,2)))
    time.sleep(1)

print("Completed 100%")

all_shows_df = pd.concat(all_shows_df, axis = 0).reset_index(drop = True)

currdate = dt.datetime.today().strftime('%Y-%m-%d')

################### SAVE TO CSV ############################
all_shows_df.to_csv('igtv_extract_{}.csv'.format(currdate))